In [1]:
from sgp4.api import Satrec
from astropy.time import Time
import numpy as np
import heyoka as hy

## Setting up the initial conditions

These are coming from a conjunction detected by SOCRATES.

In [2]:
s1 = "1 61904U 24205L   25354.90963950  .00008372  00000+0  38791-3 0  9993"
t1 = "2 61904  97.4350  65.3003 0028818 296.4783  63.3497 15.19966459 61254"

s2 = "1 62945U 97012R   25354.92518804  .00012562  00000+0  12934-2 0  9997"
t2 = "2 62945  98.3678  55.6216 0184755 142.6821 218.7391 14.78556977485038"

In [3]:
sat1 = satellite = Satrec.twoline2rv(s1, t1)
sat2 = satellite = Satrec.twoline2rv(s2, t2)

In [4]:
conj_tm = Time(val='2025-12-25 04:57:05.8814001', format='iso', scale='utc')

In [5]:
_, r1, v1 = sat1.sgp4(conj_tm.jd1, conj_tm.jd2)
_, r2, v2 = sat2.sgp4(conj_tm.jd1, conj_tm.jd2)

In [6]:
# Verify that we really are at a conjunction (g0 should be close to zero).
g0 = np.dot(np.array(r1) - r2, np.array(v1) - v2)
g0

np.float64(-5.051533271977632e-09)

## Setting up the dynamics

Here we will be using Keplerian dynamics for the two satellites, but any dynamics will work.

In [7]:
x1, y1, z1, vx1, vy1, vz1 = hy.make_vars('x1', 'y1', 'z1', 'vx1', 'vy1', 'vz1')
x2, y2, z2, vx2, vy2, vz2 = hy.make_vars('x2', 'y2', 'z2', 'vx2', 'vy2', 'vz2')
gvar = hy.make_vars('g')

state_vars = [x1, y1, z1, vx1, vy1, vz1, x2, y2, z2, vx2, vy2, vz2]

In [8]:
r1_32 = hy.sum([x1**2,y1**2,z1**2])**-1.5
r2_32 = hy.sum([x2**2,y2**2,z2**2])**-1.5

# Keplerian dynamics.
mu_earth = 3.986004415e14 / 1e9
orig_dyn = [(x1, vx1), (y1, vy1), (z1, vz1), (vx1, -x1*r1_32), (vy1, -y1*r1_32), (vz1, -z1*r1_32),
            (x2, vx2), (y2, vy2), (z2, vz2), (vx2, -x2*r2_32), (vy2, -y2*r2_32), (vz2, -z2*r2_32)]

# Add the differential equation for the event function g.
g_func = hy.sum([(x1-x2)*(vx1-vx2),(y1-y2)*(vy1-vy2),(z1-z2)*(vz1-vz2)])
dg_dt = np.dot(hy.diff_tensors([g_func], state_vars).gradient, [dyn_eq for _, dyn_eq in orig_dyn])
dyn = orig_dyn + [(gvar, dg_dt)]

In [9]:
# Create the variational integrator. We are interested in the partials wrt the state
# variables here.
vsys = hy.var_ode_sys(dyn, state_vars, order=1)
ta_var = hy.taylor_adaptive(vsys, compact_mode=True)

# Setup the initial conditions for the state variables + g.
ta_var.state[0:3] = r1
ta_var.state[3:6] = v1
ta_var.state[6:9] = r2
ta_var.state[9:12] = v2
ta_var.state[12] = g0

In [10]:
# Setup the initial conditions for the partial derivatives of g wrt the initial conditions
# of the state variables.
g_partials_cdiff = hy.cfunc(hy.diff_tensors([g_func], state_vars).gradient, state_vars)
ta_var.state[-12:] = g_partials_cdiff(list(r1)+list(v1)+list(r2)+list(v2))

In [11]:
# Evaluate the time derivative of g at t=0.
dg_dt_cfunc = hy.cfunc([dg_dt], [x1, y1, z1, vx1, vy1, vz1, x2, y2, z2, vx2, vy2, vz2])
dg_dt = dg_dt_cfunc(list(r1)+list(v1)+list(r2)+list(v2))[0]

In [12]:
# Compute the gradient of the event trigger time wrt the initial conditions
# of the state variables.
grad_te = [-_ / dg_dt for _ in ta_var.state[-12:]]

In [13]:
# Compute how the trigger time changes for small variations of the initial conditions
# of the state variables.
new_te = grad_te[0]*1e-3 - grad_te[1]*1e-3 + grad_te[2]*2e-3

In [14]:
# As a check, run a normal (i.e., non variational) numerical integration up to new_te
# with the perturbed initial conditions, and verify that we are at an event trigger.
ta = hy.taylor_adaptive(dyn, compact_mode=True)

ta.state[0:3] = r1
ta.state[3:6] = v1
ta.state[6:9] = r2
ta.state[9:12] = v2

ta.state[0] += 1e-3
ta.state[1] -= 1e-3
ta.state[2] += 2e-3

In [15]:
ta.propagate_until(new_te)

(<taylor_outcome.time_limit: -4294967299>, inf, 0.0, 1, None, None)

In [16]:
np.dot(ta.state[:3] - ta.state[6:9],ta.state[3:6] - ta.state[9:12])

np.float64(-5.051650170439029e-09)

## Propagating to the event manifold

In [17]:
ta_var

C++ datatype            : double
Tolerance               : 2.220446049250313e-16
High accuracy           : false
Compact mode            : true
Taylor order            : 20
Dimension               : 169
Time                    : 0
State                   : [565.83775274102, -1022.2727288911137, 6796.5206731389435, -2.738307601411251, -7.02945060455157, -0.8256497778812278, 565.8381115204584, -1022.2696775826441, 6796.508688714403, -3.896649570956515, -6.5515695675497225, -0.7386553441907587, -5.051533271977632e-09, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1.1583419695452641, -0.4778810370018478, -0.086

In [21]:
ta_var.step(write_tc=True, max_delta_t=0.0)

(<taylor_outcome.time_limit: -4294967299>, 0.0)

In [24]:
ta_var.propagate_until(new_te)

(<taylor_outcome.time_limit: -4294967299>, inf, 0.0, 1, None, None)

In [32]:
vstate = ta_var.eval_taylor_map([1e-3,-1e-3,2e-3,0.,0.,0.,0.,0.,0.,0.,0.,0.])

In [33]:
np.dot(vstate[:3] - vstate[6:9],vstate[3:6] - vstate[9:12])

np.float64(-5.051650170439029e-09)

In [35]:
ta_var.eval_taylor_map([1e-3,-1e-3,2e-3,0.,0.,0.,0.,0.,0.,0.,0.,0.])

array([ 5.65841291e+02, -1.02226721e+03,  6.79652344e+03, -2.73830760e+00,
       -7.02945060e+00, -8.25649778e-01,  5.65841723e+02, -1.02226361e+03,
        6.79650937e+03, -3.89664957e+00, -6.55156957e+00, -7.38655344e-01,
       -5.05153327e-09])